### Picking up failed puzzles
    1) Pull tactics from chess.com
    2) Store puzzle rating, link, time it took, if succeded/failed.

As far as I can tell, there is no chess.com API for pulling puzzles like there is for games. Should be able to parse it ourselves in some capacity from the web site.

In [ ]:
# Example: https://www.chess.com/stats/puzzles/luc777

In [2]:
import html
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

def instantiate_webdriver():
    # Set path Selenium
    CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'
    s = Service(CHROMEDRIVER_PATH)
    WINDOW_SIZE = "1920,1080"

    # Create chrome webdriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    chrome_options.add_argument('--no-sandbox')
    return webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)

In [23]:
def navigate_to_last_opened(driver):
    """Given a driver, navigate to last opened window (e.g. from a click())."""
    time.sleep(2)
    window_after = driver.window_handles[-1]
    driver.switch_to.window(window_after)
    print(driver.current_url)
    
def safe_click(driver, element):
    """Scrolls into view before clicking an element."""
    driver.execute_script("arguments[0].scrollIntoView();", element)
    element.click()
    
    
def safe_click2(driver, element):
    """Scrolls into view before clicking an element."""
    driver.execute_script("arguments[0].scrollIntoView();", element)
    driver.execute_script("arguments[0].click();", element) 


def get_user_puzzles(user, driver):
    """Find out which puzzles they recently attempted and navigate driver to that page."""
    url = f"https://www.chess.com/stats/puzzles/{user}"
    page = driver.get(url)
    time.sleep(2)
    return pd.read_html(driver.page_source)[0]

In [24]:
driver = instantiate_webdriver()

In [25]:
get_user_puzzles('Luc777', driver).head()

,Date,ID,Rating,Moves,Target,Avg Time,My Time,My Time.1,Outcome,Outcome.1
0,"Apr 24, 2022",1223479,2061,9/9,0:51,0:37,0:56,99%,10,2439
1,"Apr 24, 2022",1128162,2020,4/6,0:35,1:52,0:47,64%,5,2429
2,"Apr 24, 2022",1310551,2274,2/4,0:38,1:21,1:21,44%,-4,2424
3,"Apr 24, 2022",1669636,2207,3/3,0:28,0:55,0:31,99%,11,2428
4,"Apr 24, 2022",1835548,2281,4/4,0:34,1:28,2:42,70%,5,2417


### Navigate to tab-rush

In [29]:
def navigate_to_puzzle_rush_table(user, driver):
    # Navigate to their puzzles page
    url = f"https://www.chess.com/stats/puzzles/{user}"
    page = driver.get(url)
    time.sleep(2)
    # Click the rush tab
    element = driver.find_element_by_id("tab-rush")
    time.sleep(2)
    driver.execute_script("arguments[0].scrollIntoView();", element)
    element.click()

In [30]:
def get_user_puzzle_rush_scores(user, driver):
    navigate_to_puzzle_rush_table(user, driver)
    return pd.read_html(driver.page_source)[0]

In [32]:
df_rush = get_user_puzzle_rush_scores("Luc777", driver)
df_rush.head(3)

,Date,Time,Result,Streak,Highest Solved,Time/Puzzle
0,"Apr 24, 2022",5 min,24,17,1450,0:11
1,"Apr 24, 2022",5 min,22,21,1276,0:13
2,"Apr 17, 2022",5 min,22,22,1246,0:12


### Now hone in on specific puzzles missed in their rush
This assumes the driver is on a user's puzzle page + has clicked the "rush" tab. 

Navigate to the page describing the most recent rush

In [33]:
from bs4 import BeautifulSoup
def element_to_failed_puzzles_dataset(element, driver):
    """ Given an element pointing to a specific puzzle rush, generate an array describing 
    failed puzzles.
    """
    # Navigate to specific puzzle rush page
    element.click()
    navigate_to_last_opened(driver)
    source_rush = driver.current_url
    source_rush_window = driver.window_handles[-1]
    
    # Get failed puzzles
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    puzzle_divs = soup.find_all("div", {"class": "streak-indicator-streak streak-indicator-incorrect streak-indicator-link"})
    puzzle_ratings = [puzzle_div.text.strip() for puzzle_div in puzzle_divs]
    failure_links = driver.find_elements_by_class_name('streak-indicator-incorrect')
    
    # Iterate through failed puzzles to create dataset
    failed_puzzle_dataset = []
    for puzzle_rating, failure_element in zip(puzzle_ratings, failure_links):
        # Navigate to each failed puzzle to get link (hacky)
        time.sleep(3)
        print(failure_element)
        #safe_click(driver, failure_element)
        safe_click2(driver, failure_element)
        navigate_to_last_opened(driver)
        failed_puzzle_dataset.append((puzzle_rating, driver.current_url, source_rush))
        # Switch back to source
        time.sleep(1)
        driver.switch_to.window(source_rush_window)
    return failed_puzzle_dataset

In [39]:
def get_failed_puzzles(driver, user, limit=10):
    """Generate a list of failed puzzles from Puzzle Rush."""
    navigate_to_puzzle_rush_table(user, driver)
    # Elements of rows (tr), where each is a separate puzzle rush. 
    elmnts = driver.find_elements_by_xpath("//table/tbody/tr")
    failed_puzzles = []
    for i in range(min(len(elmnts), 10)):
        navigate_to_puzzle_rush_table(user, driver)
        elmnts = driver.find_elements_by_xpath("//table/tbody/tr")
        failed_puzzles.extend(element_to_failed_puzzles_dataset(elmnts[i], driver))
    return failed_puzzles

In [41]:
driver = instantiate_webdriver()
failed_puzzles_data = get_failed_puzzles(driver, "Luc777")

https://www.chess.com/puzzles/rush/luc777/6Q6WA
<selenium.webdriver.remote.webelement.WebElement (session="35e38053ae690ecccdc18eb94572f6c9", element="85344635-8e7c-4ae9-80bc-ffb3cb53255d")>
https://www.chess.com/puzzles/problem/621340
<selenium.webdriver.remote.webelement.WebElement (session="35e38053ae690ecccdc18eb94572f6c9", element="c27c3f62-7bd3-4d2b-888e-515b6822aa5a")>
https://www.chess.com/puzzles/problem/1150640
https://www.chess.com/puzzles/rush/luc777/4XoRuY
<selenium.webdriver.remote.webelement.WebElement (session="35e38053ae690ecccdc18eb94572f6c9", element="9b32d7c3-b244-40b5-bbee-951b7710c9e8")>
https://www.chess.com/puzzles/problem/1644026
https://www.chess.com/puzzles/rush/luc777/Q9iN2
<selenium.webdriver.remote.webelement.WebElement (session="35e38053ae690ecccdc18eb94572f6c9", element="69e8d6bc-c06d-47a5-9f91-7cb152e3bcbe")>
https://www.chess.com/puzzles/problem/1198420
<selenium.webdriver.remote.webelement.WebElement (session="35e38053ae690ecccdc18eb94572f6c9", elemen

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: headless chrome=99.0.4844.84)


Moved this code to chess_analytics/puzzle_fetcher.py.